# SHIFT-23. Answers to questions on data
# 5. Uplift for clients with time since last purchase >= 10 days

In [1]:
import pandas as pd

I assume 10 days between the client's last purchase and the last date in train_purch

In [2]:
train_purch = pd.read_csv('train_purch.csv',
                       parse_dates=['transaction_datetime'])
train_purch

,client_id,transaction_id,transaction_datetime,regular_points_received,express_points_received,regular_points_spent,express_points_spent,purchase_sum,store_id,product_id,product_quantity,trn_sum_from_iss,trn_sum_from_red
0,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,9a80204f78,2.0,80.0,NaN
1,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,da89ebd374,1.0,65.0,NaN
2,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,0a95e1151d,1.0,24.0,NaN
3,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,4055b15e4a,2.0,50.0,NaN
4,000012768d,7e3e2e3984,2018-12-01 07:12:45,10.0,0.0,0.0,0.0,1007.0,54a4a11a29,a685f1916b,1.0,22.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
15998947,fffeca6d22,a0bb11a968,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,a0613e36b5,de437430e2,1.0,19.0,NaN
15998948,fffeca6d22,a0bb11a968,2019-03-09 07:47:43,0.4,0.0,0.0,0.0,99.0,a0613e36b5,ad8fee4200,1.0,45.0,NaN
15998949,fffeca6d22,3b876a5532,2019-03-15 14:46:39,0.8,0.0,0.0,0.0,165.0,a0613e36b5,ad8fee4200,2.0,94.0,NaN
15998950,fffeca6d22,3b876a5532,2019-03-15 14:46:39,0.8,0.0,0.0,0.0,165.0,a0613e36b5,230dbb0c16,1.0,50.0,NaN


In [3]:
last_day = train_purch.transaction_datetime.max()
last_day

Timestamp('2019-03-18 23:19:28')

In [7]:
last_purch = train_purch.groupby('client_id')\
    .agg({'transaction_datetime': 'max'})\
    .rename(columns={'transaction_datetime': 'last_purch'})
last_purch

,last_purch
client_id,
000012768d,2019-03-14 15:01:47
000036f903,2019-03-17 10:29:37
0001f552b0,2019-03-16 10:54:02
00020e7b18,2019-03-15 12:04:23
000220a0a7,2019-03-12 16:44:43
...,...
fffd5cd0c6,2019-03-18 15:17:33
fffd63dfe3,2019-03-13 08:54:48
fffd8c9d7d,2019-03-08 13:06:17


In [14]:
not_recent = (last_day - last_purch >= pd.Timedelta(10, 'd'))\
    .rename(columns={'last_purch': 'not_recent'})
not_recent

,not_recent
client_id,
000012768d,False
000036f903,False
0001f552b0,False
00020e7b18,False
000220a0a7,False
...,...
fffd5cd0c6,False
fffd63dfe3,False
fffd8c9d7d,True


In [18]:
df = pd.read_csv('train.csv', index_col='client_id')\
    .loc[not_recent.not_recent, :]
df

,treatment_flg,purchased
client_id,,
7c1ccbf93f,1,1
0c11f1cc19,1,0
376f3823f1,1,0
63adeb36fb,1,1
3c1f3258d9,0,1
...,...,...
780fdce344,1,0
032543f353,0,0
bb2080ac70,0,1


In [19]:
treat = df.loc[df.treatment_flg == 1, 'purchased']\
    .value_counts(normalize=True)
treat

0    0.579912
1    0.420088
Name: purchased, dtype: float64

In [20]:
control = df.loc[df.treatment_flg == 0, 'purchased']\
    .value_counts(normalize=True)
control

0    0.617154
1    0.382846
Name: purchased, dtype: float64

In [21]:
uplift = treat[1] - control[1]
uplift

0.03724156974927306